In [1]:
from bert.utils import obtain_sentence_embeddings
from data.utils import load_training_dictionaries
from extractor.utils import ExtractorModel
from extractor.train import get_training_batch
from pytorch_transformers import BertModel
from pytorch_transformers import BertTokenizer

import torch
import numpy as np

In [2]:
# Load data:
model = ExtractorModel()
model_path = "results/models/extractor.pt"
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [3]:
data = load_training_dictionaries()
documents, extraction_labels = get_training_batch(data, batch_size=10)

In [4]:
sentence_embeddings, mask = obtain_sentence_embeddings(model.bert_model, model.bert_tokenizer, documents)

In [5]:
extraction_probabilities = model(sentence_embeddings)
extraction_probabilities

tensor([[5.9915e-04, 9.9328e-01, 5.3232e-04, 5.4050e-04, 5.6455e-04, 9.9860e-01,
         9.9871e-01, 5.4179e-04, 5.1287e-04, 6.5203e-04, 7.0618e-04, 5.4396e-04,
         5.4196e-04, 5.9824e-04, 5.6216e-04, 6.2982e-04, 5.6603e-04, 5.6209e-04,
         8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01,
         8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01,
         8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01,
         8.3428e-01, 8.3428e-01, 8.3428e-01],
        [9.9881e-01, 9.9775e-01, 5.1092e-04, 5.4794e-04, 5.1083e-04, 9.9721e-01,
         5.2902e-04, 5.1224e-04, 5.2280e-04, 5.1305e-04, 5.2701e-04, 5.1188e-04,
         5.1340e-04, 5.1789e-04, 9.9666e-01, 5.2543e-04, 6.3789e-04, 9.6206e-01,
         9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01,
         9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01,
         9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337

In [6]:
max_extracted_sentences = int(torch.max(torch.tensor([x.sum() for x in extraction_labels])))
vals, predicted_idx = torch.topk((extraction_probabilities * mask), k=max_extracted_sentences, dim=1)
vals

tensor([[9.9871e-01, 9.9860e-01, 9.9328e-01, 7.0618e-04],
        [9.9881e-01, 9.9775e-01, 9.9721e-01, 9.9666e-01],
        [9.9866e-01, 9.9580e-01, 5.5258e-04, 5.2462e-04],
        [9.9902e-01, 9.9828e-01, 9.9806e-01, 5.5226e-04],
        [9.9894e-01, 9.9876e-01, 5.3652e-04, 5.1733e-04],
        [9.9823e-01, 9.9809e-01, 5.7799e-04, 5.6243e-04],
        [9.9861e-01, 9.9774e-01, 5.4555e-04, 5.2975e-04],
        [9.9899e-01, 9.9816e-01, 9.9814e-01, 5.3069e-04],
        [9.9859e-01, 9.9566e-01, 5.5375e-04, 5.3051e-04],
        [9.9828e-01, 9.9812e-01, 9.9730e-01, 1.1658e-03]],
       grad_fn=<TopkBackward>)

In [7]:
for i, doc in enumerate(documents):
    label_idx = extraction_labels[i].numpy()[:len(doc)]
    label_idx = label_idx.astype(bool)
    
    n_summary_sentences = label_idx.sum()
    
    target = np.array(doc)[label_idx]
    prediction = np.array(doc)[predicted_idx[i]][:n_summary_sentences]
    
    print("\n\n".join(target.tolist()))
    print("\n\n---------------------------\n\n")
    print("\n\n".join(prediction.tolist()))
    
    print("\n\n-----------------------------------------------------")
    print("-----------------------------------------------------")
    print("-----------------------------------------------------\n\n")


one french citizen , one belgian and three malians were killed in the attack in the capital of bamako , said gabriel toure , director of a local hospital .

a north african jihadist group , al - murabitun , claimed responsibility for the attack , according to mauritanian news agency al akhbar . the purported claim came in an audio message in which the group said it carried out the attack in retaliation for the killing of one of its leaders , al akhbar said .

" al - murabitun may be considered a regional competitor to al - qaeda in the islamic maghreb ( aqim ) , " according to the jamestown foundation , a washington - based research and analysis firm . the u.s. state department said in january that al - murabitun is a " newly - formed " militant group that has presence in northern mali .


---------------------------


" al - murabitun may be considered a regional competitor to al - qaeda in the islamic maghreb ( aqim ) , " according to the jamestown foundation , a washington - based r

In [52]:
input_sentences = ["Hello my name is Andrew", "roger is cool"]


torch.Size([2, 5, 768])

In [115]:
a = torch.randn((2, 1, 4))
print(a.shape)
print()
print(torch.argmax(a, dim=2, keepdim=True).shape)
print(torch.argmax(a, dim=2, keepdim=True))
print()
print(torch.argmax(a, dim=2).shape)
print(torch.argmax(a, dim=2))

torch.Size([2, 1, 4])

torch.Size([2, 1, 1])
tensor([[[3]],

        [[2]]])

torch.Size([2, 1])
tensor([[3],
        [2]])


tensor([[3],
        [2]])

In [126]:
torch.argmax(a, dim=2).shape

torch.Size([2, 1])

In [125]:
bert_model(torch.argmax(a, dim=2))[0].shape

torch.Size([2, 1, 768])

In [63]:
a = torch.tensor([
    [1, 2, 3], 
    [4, 5, 6], 
    [7, 8, 9]
])

print(a)

row = tuple(torch.tensor([0, 1]))
columns = tuple(torch.tensor([0, 2]))
a[row, columns]

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


tensor([1, 6])